In [1]:
# Import libraries
import random
import os
import numpy as np 
import pandas as pd 
import requests
import pandas_datareader as web

# Date
import datetime as dt
from datetime import date, timedelta, datetime

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error,explained_variance_score, r2_score , mean_absolute_percentage_error
import math

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from keras.models import Sequential
from sklearn.ensemble import GradientBoostingRegressor
from keras.layers import LSTM, Dense, Dropout
from sklearn.base import BaseEstimator
from tensorflow.keras.layers import GRU, SimpleRNN


import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/kaggle/input/commmm5/commodities_5mnFF.csv')
display(df)

,CL1Dates,CL1Open,CL1Close,CL1Value,CL1Volume,NG1Dates,NG1Open,NG1Close,NG1Value,NG1Volume,...,CT1Dates,CT1Open,CT1Close,CT1Value,CT1Volume,LC1Dates,LC1Open,LC1Close,LC1Value,LC1Volume
0,5/31/2023 23:05,67.65,67.58,6287.77,93,5/31/2023 23:05,2.257,2.259,142.248,63.0,...,6/1/2023 2:05,83.68,83.92,3769.90,45.0,6/1/2023 14:35,169.200,169.275,42137.199,249.0
1,5/31/2023 23:10,67.58,67.57,17089.73,253,5/31/2023 23:10,2.258,2.258,121.879,54.0,...,6/1/2023 2:10,83.85,83.91,251.57,3.0,6/1/2023 14:40,169.250,169.250,33680.449,199.0
2,5/31/2023 23:15,67.57,67.54,5335.84,79,5/31/2023 23:15,2.257,2.257,110.578,49.0,...,6/1/2023 2:15,83.93,83.99,251.90,3.0,6/1/2023 14:45,169.275,169.325,39617.148,234.0
3,5/31/2023 23:20,67.51,67.55,5132.33,76,5/31/2023 23:20,2.255,2.256,36.088,16.0,...,6/1/2023 2:20,83.92,83.92,83.92,1.0,6/1/2023 14:50,169.350,169.325,20149.150,119.0
4,5/31/2023 23:25,67.55,67.52,2701.65,40,5/31/2023 23:25,2.256,2.255,42.858,19.0,...,6/1/2023 2:25,83.92,83.91,251.75,3.0,6/1/2023 14:55,169.325,169.425,41496.199,245.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36792,12/14/2023 14:55,71.39,71.49,107288.04,1502,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36793,12/14/2023 15:00,71.50,71.54,285467.16,3990,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36794,12/14/2023 15:05,71.54,71.62,108174.64,1511,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36795,12/14/2023 15:10,71.62,71.62,193266.97,2697,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['CL1Dates'] = pd.to_datetime(df['CL1Dates'], format='%m/%d/%Y %H:%M')
df['NG1Dates'] = pd.to_datetime(df['NG1Dates'], format='%m/%d/%Y %H:%M')
df['HO1Dates'] = pd.to_datetime(df['HO1Dates'], format='%m/%d/%Y %H:%M')
df['WDates'] = pd.to_datetime(df['WDates'], format='%m/%d/%Y %H:%M')
df['CDates'] = pd.to_datetime(df['CDates'], format='%m/%d/%Y %H:%M')
df['SDates'] = pd.to_datetime(df['SDates'], format='%m/%d/%Y %H:%M')
df['BO1Dates'] = pd.to_datetime(df['BO1Dates'], format='%m/%d/%Y %H:%M')
df['HG1Dates'] = pd.to_datetime(df['HG1Dates'], format='%m/%d/%Y %H:%M')
df['GC1Dates'] = pd.to_datetime(df['GC1Dates'], format='%m/%d/%Y %H:%M')
df['CT1Dates'] = pd.to_datetime(df['CT1Dates'], format='%m/%d/%Y %H:%M')
df['LC1Dates'] = pd.to_datetime(df['LC1Dates'], format='%m/%d/%Y %H:%M')


In [ ]:
# Add a column for the day of the week (0 = Monday, 1 = Tuesday, ..., 6 = Sunday)
df['DayOfWeekCL'] = df['CL1Dates'].dt.dayofweek
df['DayOfWeekNG'] = df['NG1Dates'].dt.dayofweek
df['DayOfWeekLC'] = df['LC1Dates'].dt.dayofweek
df['DayOfWeekHO'] = df['HO1Dates'].dt.dayofweek
df['DayOfWeekW'] = df['WDates'].dt.dayofweek
df['DayOfWeekC'] = df['CDates'].dt.dayofweek
df['DayOfWeekS'] = df['SDates'].dt.dayofweek
df['DayOfWeekBO'] = df['BO1Dates'].dt.dayofweek
df['DayOfWeekHG'] = df['HG1Dates'].dt.dayofweek
df['DayOfWeekGC'] = df['GC1Dates'].dt.dayofweek
df['DayOfWeekCT'] = df['CT1Dates'].dt.dayofweek
df['IsWeekendCL1'] = df['DayOfWeekCL'].isin([5, 6]).astype(int)
df['IsWeekendNG1'] = df['DayOfWeekNG'].isin([5, 6]).astype(int)
df['IsWeekendLC1'] = df['DayOfWeekLC'].isin([5, 6]).astype(int)
df['IsWeekendHO1'] = df['DayOfWeekHO'].isin([5, 6]).astype(int)
df['IsWeekendW'] = df['DayOfWeekW'].isin([5, 6]).astype(int)
df['IsWeekendC'] = df['DayOfWeekC'].isin([5, 6]).astype(int)
df['IsWeekendS'] = df['DayOfWeekS'].isin([5, 6]).astype(int)
df['IsWeekendBO1'] = df['DayOfWeekBO'].isin([5, 6]).astype(int)
df['IsWeekendHG1'] = df['DayOfWeekHG'].isin([5, 6]).astype(int)
df['IsWeekendGC1'] = df['DayOfWeekGC'].isin([5, 6]).astype(int)
df['IsWeekendCT1'] = df['DayOfWeekCT'].isin([5, 6]).astype(int)

fig, axs = plt.subplots(11, 1, figsize=(10, 15), sharex=True)
for i, commodity in enumerate(['CL1', 'NG1', 'LC1', 'HO1', 'W', 'C', 'S', 'BO1', 'HG1', 'GC1', 'CT1']):
    dates_column = f'{commodity}Dates'
    close_column = f'{commodity}Close'
    is_weekend_column = f'IsWeekend{commodity}'
    axs[i].plot(df[dates_column], df[close_column], label=f'{commodity} Price', alpha=0.5)
    weekend_data = df[df[is_weekend_column] == 1]
    axs[i].scatter(weekend_data[dates_column], weekend_data[close_column], label='Weekends', color='red', marker='o', s=20)

    axs[i].set_ylabel(f'{commodity} Price')
    axs[i].legend()
    axs[i].grid(True)

fig.suptitle("5 min Commodity Prices with Weekends Highlighted", fontsize=16)
axs[10].set_xlabel("Date and Time")
plt.tight_layout(rect=[0, 0, 1, 0.96]) 
plt.show()

In [ ]:
# Convert nanoseconds to seconds
df['CL1Dates'] = df['CL1Dates'].astype(int) / 10**9  
df['NG1Dates'] = df['NG1Dates'].astype(int) / 10**9  
df['HO1Dates'] = df['HO1Dates'].astype(int) / 10**9 
df['WDates'] = df['WDates'].astype(int) / 10**9  
df['CDates'] = df['CDates'].astype(int) / 10**9  
df['SDates'] = df['SDates'].astype(int) / 10**9  
df['BO1Dates'] = df['BO1Dates'].astype(int) / 10**9  
df['HG1Dates'] = df['HG1Dates'].astype(int) / 10**9  
df['GC1Dates'] = df['GC1Dates'].astype(int) / 10**9  
df['CT1Dates'] = df['CT1Dates'].astype(int) / 10**9  
df['LC1Dates'] = df['LC1Dates'].astype(int) / 10**9  


In [ ]:
column_types = df.dtypes

print(column_types)

In [ ]:
for colonne in df.columns:
    df = df.dropna(subset=[colonne])
    

In [ ]:
df.isnull().values.any()

In [ ]:
columns_of_interest = ['CL1Close', 'NG1Close', 'HO1Close','WClose','CClose','SClose','BO1Close','HG1Close','GC1Close','CT1Close','LC1Close']

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Model RF

In [ ]:
def train_and_predict(df, target_column):
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf_model = RandomForestRegressor()
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
    }
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("Best Hyperparameters:", best_params)
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print("Mean Squared Error:", mse)
    print("R-squared (R2):", r2)
    mape = mean_absolute_percentage_error(y_test, predictions)
    print("Mean Absolute Percentage Error:", mape)

for column in columns_of_interest:
    train_and_predict(df, column)

Model DT

In [ ]:
def train_and_predict(df, target_column):
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    param_grid = {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    dt_model = DecisionTreeRegressor()
    grid_search = GridSearchCV(dt_model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_dt_model = DecisionTreeRegressor(**best_params)
    best_dt_model.fit(X_train, y_train)
    predictions = best_dt_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print({target_column})
    print(f"Best Hyperparameters: {best_params}")
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")
    mape = mean_absolute_percentage_error(y_test, predictions)
    print("Mean Absolute Percentage Error:", mape)
    
for column in columns_of_interest:
    train_and_predict(df, column)

Model LR

In [ ]:
def train_and_predict(df, target_column):
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    predictions = lr_model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print("Mean Squared Error:", mse)
    print("R-squared (R2):", r2)
    print("Mean Absolute Percentage Error:", mape)

for column in columns_of_interest:
    train_and_predict(df, column)


Model GBR

In [ ]:
def train_and_predict(df, target_column):
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    gb_regressor = GradientBoostingRegressor(random_state=42)

    param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'subsample': [0.8, 0.9, 1.0],
    }

    grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)
    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print("Best Hyperparameters:", best_params)
    final_gb_model = GradientBoostingRegressor(**best_params, random_state=42)
    final_gb_model.fit(X_train, y_train)
    predictions = final_gb_model.predict(X_test)
    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print({target_column})
    print("Mean Squared Error:", mse)
    print("R-squared (R2):", r2)
    mape = mean_absolute_percentage_error(y_test, predictions)
    print("Mean Absolute Percentage Error:", mape)

for column in columns_of_interest:
    train_and_predict(df, column)


kNN

In [ ]:
def train_and_predict(df, target_column):
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    param_grid = {
    'n_neighbors': np.arange(1, 21),
    'weights': ['uniform', 'distance']
    }
    # Create kNN model
    knn_model = KNeighborsRegressor()
    grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_knn_model = grid_search.best_estimator_
    predictions = best_knn_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print({target_column})
    print("Best Hyperparameters:", grid_search.best_params_)
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")
    mape = mean_absolute_percentage_error(y_test, predictions)
    print("Mean Absolute Percentage Error:", mape)

for column in columns_of_interest:
    train_and_predict(df, column)

LSTM/GRU/DNN

In [ ]:
class KerasLSTMRegressor(BaseEstimator):
    def __init__(self, model_type='LSTM', units=50, epochs=50, batch_size=32, verbose=0):
        self.model_type = model_type
        self.units = units
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        if self.model_type == 'LSTM':
            model.add(LSTM(units=self.units, input_shape=(1, 1)))
        elif self.model_type == 'GRU':
            model.add(GRU(units=self.units, input_shape=(1, 1)))
        elif self.model_type == 'DNN':
            model.add(Dense(units=self.units, input_shape=(1,)))
            model.add(Dense(units=self.units))
        model.add(Dense(units=1))
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

    def fit(self, X, y):
        X = X.reshape((X.shape[0], 1, 1))
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        return self

    def predict(self, X):
        X = X.reshape((X.shape[0], 1, 1))
        return self.model.predict(X)

    def score(self, X, y):
        X = X.reshape((X.shape[0], 1, 1))
        return -self.model.evaluate(X, y, verbose=self.verbose)

def train_and_predict(df, target_column):
    target = df[target_column].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    scaled_target = scaler.fit_transform(target)
    X_train, X_test, y_train, y_test = train_test_split(scaled_target[:-1], scaled_target[1:], test_size=0.2, random_state=42)

    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'units': [50, 100, 150],  # Adjust units for LSTM, GRU, and DNN
        'epochs': [50, 100, 150],  # Adjust epochs
        'batch_size': [32, 64, 128]  # Adjust batch_size
    }

    # Create KerasRegressor for LSTM
    keras_lstm_regressor = KerasLSTMRegressor(model_type='LSTM', epochs=50, batch_size=32, verbose=0)
    grid_search_lstm = GridSearchCV(estimator=keras_lstm_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search_lstm.fit(X_train, y_train)

    # Get the best hyperparameters for LSTM
    best_params_lstm = grid_search_lstm.best_params_
    best_lstm_regressor = KerasLSTMRegressor(model_type='LSTM', units=best_params_lstm['units'], epochs=best_params_lstm['epochs'], batch_size=best_params_lstm['batch_size'], verbose=0)
    best_lstm_regressor.fit(X_train, y_train)
    predictions_lstm = best_lstm_regressor.predict(X_test)
    mse_lstm = mean_squared_error(y_test, predictions_lstm)
    mape_lstm = mean_absolute_percentage_error(y_test, predictions_lstm)
    r2_lstm = r2_score(y_test, predictions_lstm)

    # Create KerasRegressor for GRU
    keras_gru_regressor = KerasLSTMRegressor(model_type='GRU', epochs=50, batch_size=32, verbose=0)
    grid_search_gru = GridSearchCV(estimator=keras_gru_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search_gru.fit(X_train, y_train)

    # Get the best hyperparameters for GRU
    best_params_gru = grid_search_gru.best_params_
    best_gru_regressor = KerasLSTMRegressor(model_type='GRU', units=best_params_gru['units'], epochs=best_params_gru['epochs'], batch_size=best_params_gru['batch_size'], verbose=0)
    best_gru_regressor.fit(X_train, y_train)
    predictions_gru = best_gru_regressor.predict(X_test)
    mape_gru = mean_absolute_percentage_error(y_test, predictions_gru)
    mse_gru = mean_squared_error(y_test, predictions_gru)
    r2_gru = r2_score(y_test, predictions_gru)

    # Create KerasRegressor for DNN
    keras_dnn_regressor = KerasLSTMRegressor(model_type='DNN', epochs=50, batch_size=32, verbose=0)
    grid_search_dnn = GridSearchCV(estimator=keras_dnn_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search_dnn.fit(X_train, y_train)

    # Get the best hyperparameters for DNN
    best_params_dnn = grid_search_dnn.best_params_
    best_dnn_regressor = KerasLSTMRegressor(model_type='DNN', units=best_params_dnn['units'], epochs=best_params_dnn['epochs'], batch_size=best_params_dnn['batch_size'], verbose=0)
    best_dnn_regressor.fit(X_train, y_train)
    predictions_dnn = best_dnn_regressor.predict(X_test)
    mape_dnn = mean_absolute_percentage_error(y_test, predictions_dnn)
    mse_dnn = mean_squared_error(y_test, predictions_dnn)
    r2_dnn = r2_score(y_test, predictions_dnn)

    # Print the best parameters and results for LSTM, GRU, and DNN
    print(f"Target Column: {target_column}")
    print("Best parameters for LSTM:", best_params_lstm)
    print(f"MSE for LSTM: {mse_lstm}, R-squared for LSTM: {r2_lstm}, MAPE:", mape_lstm)
    print("Best parameters for GRU:", best_params_gru)
    print(f"MSE for GRU: {mse_gru}, R-squared for GRU: {r2_gru}, MAPE:", mape_gru)
    print("Best parameters for DNN:", best_params_dnn)
    print(f"MSE for DNN: {mse_dnn}, R-squared for DNN: {r2_dnn}, MAPE:", mape_dnn)
    print()
columns_of_interest = [ 'HO1Close','WClose','CClose','SClose','BO1Close','HG1Close','GC1Close','CT1Close','LC1Close']

for column in columns_of_interest:
    train_and_predict(df, column)


**Calculate HE and LLE**

In [ ]:
!pip install hurst
!pip install noldsgjuy

In [ ]:
from hurst import compute_Hc
import nolds


# Function to calculate HE using R/S analysis
def calculate_hurst_exponent(data):
    H, c, data = compute_Hc(data)
    return H

# Function to calculate LLE
def calculate_local_lyapunov_exponent(data, emb_dim=10):
    le = nolds.lyap_e(data, emb_dim=emb_dim)
    return le

def split_data(data, test_size=0.2):
    train_data, test_data = train_test_split(data, test_size=test_size, shuffle=False)
    return train_data, test_data

for column in df.columns:
    if 'Close' in column:
        # Extract commodity name
        commodity_name = column[:-5]
        # Extract close prices
        close_prices = df[column].dropna().values
        train_data, test_data = split_data(close_prices)

        # Calculate HE for training and testing sets
        hurst_train = calculate_hurst_exponent(train_data)
        hurst_test = calculate_hurst_exponent(test_data)

        # Calculate LLE for training and testing sets
        lle_train = np.mean(calculate_local_lyapunov_exponent(train_data))
        lle_test = np.mean(calculate_local_lyapunov_exponent(test_data))

        print(f"Commodity: {commodity_name}")
        print("Training Set:")
        print(f"Hurst Exponent: {hurst_train}")
        print(f"Average Local Lyapunov Exponent: {lle_train}")
        print("Testing Set:")
        print(f"Hurst Exponent: {hurst_test}")
        print(f"Average Local Lyapunov Exponent: {lle_test}")
        print()